In [29]:
from bokeh.io import show, output_notebook
from bokeh.layouts import column,row
output_notebook(hide_banner=True)
from cocoa import covid19 as co
from cocoa import display as cd
from cocoa import world as cw
import pandas as pd
w=cw.WorldInfo()

# SPF database choosen 

In [44]:
spf = co.DataBase('spf')
coco = cd.CocoDisplay()

SPF aka Sante Publique France database selected ...
... tree differents db from SPF will be parsed ...
Available keys words for  spf  are :  ['hosp', 'rea', 'rad', 'dc', 'incid_hosp', 'incid_rea', 'incid_dc', 'incid_rad', 'P', 'T', 'tx_incid', 'R', 'taux_occupation_sae', 'tx_pos']
Last date data  10/14/20


In [3]:
sp=spf.get_stats(which='incid_rea',location=w.getIleDeFrance()[0])
show(coco.cocoa_basic_plot([sp],['incid_rea']))

In [4]:
sp=spf.get_stats(which='incid_dc',location=w.getIleDeFrance()[0])
show(coco.cocoa_basic_plot([sp],['incid_dc']))

In [5]:
show(coco.cocoa_basic_plot([sp],['cumul']))

In [6]:
sp=spf.get_stats(which='dc',location=w.getIleDeFrance()[0])
show(coco.cocoa_basic_plot([sp],['dc']))

In [27]:
sp_dc=spf.get_stats(which='dc',location='13')
sp_cumul=spf.get_stats(which='incid_dc',location='13')
sp_dc['cumul_incid_dc'] = sp_cumul['cumul'] 
show(coco.cocoa_basic_plot([sp_dc],['dc','cumul_incid_dc']))

In [30]:
def retrieve_R(loc):
    sp=spf.get_stats(which='R',location=loc)
    sp = sp.groupby('date').agg({'R': "mean"})
    sp = sp.reset_index()
    df = sp[['date', 'R']]
    df = df.dropna()
    return df.set_index('date')

location = w.getAllFranceDepartment()

a = spf.get_stats(which='incid_hosp',location=location,output='pandas')
a = a.groupby('date').agg({'incid_hosp': "sum"})

smoothed = spf.smooth_cases(a.stack())
posteriors = spf.get_posteriors(smoothed)
posteriors = posteriors.idxmax()
a['Reff'] =  posteriors.unstack()['incid_hosp']


df = retrieve_R(location)
result=pd.concat([a,df], axis=1, sort=False)
fig=coco.cocoa_basic_plot([result.dropna()],['Reff','R'])
show(fig)

# Function to resume data

In [34]:
def resumeplot(loc):
    if type(loc) is list:
        location = loc
        name = 'France'
    if type(loc) is str:
        location = loc
        name = loc
    if type(loc) is tuple: 
        location = loc[0]
        name = loc[1]
    
    sp=spf.get_stats(which='rea',location=location)
    sp = sp.groupby('date').agg({'rea': "sum"})
    sp['rea'] = sp['rea'].rolling(7).mean()
    fig1=coco.cocoa_basic_plot([sp],['rea'],title='Rea ' + name +' moyenne glissante sur 7 jours')

    dc=spf.get_stats(which='incid_dc',location=location)
    dc = dc.groupby('date').agg({'incid_dc': "sum"})
    dc['incid_dc'] = dc['incid_dc'].rolling(7).mean()
    fig2=coco.cocoa_basic_plot([dc],['incid_dc'],title='Deces (hopital) ' + name +' moyenne glissante sur 7 jours')
    
    po=spf.get_stats(which='P',location=location)
    po = po.groupby('date').agg({'P': "sum"})
    po['P'] = po['P'].rolling(7).mean()

    te=spf.get_stats(which='T',location=location)
    te = te.groupby('date').agg({'T': "sum"})
    te['T'] = te['T'].rolling(7).mean()
    po['positivite'] = po['P']/te['T'] 
    fig3=coco.cocoa_basic_plot([po],['positivite'],title='Taux Positivité ' + name +' moyenne glissante sur 7 jours')
    
    ta=spf.get_stats(which='taux_occupation_sae',location=location)
    ta = ta.groupby('date').agg({'taux_occupation_sae': "mean"})
    ta['taux_occupation_sae'] = ta['taux_occupation_sae'].rolling(7).mean()
    fig4=coco.cocoa_basic_plot([ta],['taux_occupation_sae'],title='nombre de lits de réanimation occupés par les patients atteints du COVID-19 sur la capacité initiale en réanimation' + name +' moyenne glissante sur 7 jours')
 
    show(column(row(fig1,fig2),row(fig3,fig4)))

# Admissions en réa moyenne glissante sur 7 jours, toute la France
(identique à article de catherine hill )

In [35]:
resumeplot(w.getAllFranceDepartment())

# Admissions en réa Ile-de-France, moyenne glissante sur 7 jours

In [36]:
resumeplot(w.getIleDeFrance())

In [37]:
resumeplot(w.getProvenceAlpesCotedAzur())

In [38]:
resumeplot(w.getBretagne())

# Statistiques hospitalières

Île-de-France  

In [39]:
from datetime import datetime, timedelta
loc = w.getIleDeFrance()[0]
rea = spf.get_stats(which='incid_rea',location=loc)
#max(rea['date']
mask1 = (rea['date'] == '2020-10-08')
mask2 = (rea['date'] == '2020-10-07') # - timedelta(days=1))
reacum1 = rea.loc[mask1]
reacum2 = rea.loc[mask2]
print("Réanimations au total depuis le début de l'épidémie :",sum(reacum1['cumul']))
print("en 24h :", sum(reacum1['cumul']) - sum(reacum2['cumul']))

Réanimations au total depuis le début de l'épidémie : 8371.0
en 24h : 28.0


In [41]:
rea=rea.groupby('date').agg({'incid_rea': "sum"})
show(coco.cocoa_basic_plot([rea],['incid_rea']))

Hospitalisations au total depuis le début de l'épidémie 

In [42]:
loc = w.getIleDeFrance()[0]
hosp = spf.get_stats(which='incid_hosp',location=loc)
mask1 = (hosp['date'] == '2020-10-08')
mask2 = (hosp['date'] == '2020-10-07')
hospcum1 = hosp.loc[mask1]
hospcum2 = hosp.loc[mask2]

hospcum = hosp.loc[mask1]
print("Hospitalisations au total depuis le début de l'épidémie :",sum(hospcum['cumul']))
print("en 24h :", sum(hospcum1['cumul'])-sum(hospcum2['cumul']))

Hospitalisations au total depuis le début de l'épidémie : 49324.0
en 24h : 201.0


# Reff France (méthode de Cori) 
https://covidtracker.fr/covidtracker-france/

Méthode de calcul basée sur la méthode de Cori :
R = A / B

A = nb. de personnes admises aux urgences pour Covid19 (sur 7 jours)
B = nb. de personnes admises aux urgences pour Covid19 il y a 7 jours (sur 7 jours)

In [43]:
location = w.getAllFranceDepartment()
a = spf.get_stats(which='incid_hosp',location=location,output='pandas')
a = a.groupby('date').agg({'incid_hosp': "sum"})
a['incid_hosp']=a['incid_hosp'].rolling(7).mean()
a['incid_hosp7']= a['incid_hosp'].shift(7)

a['ratio'] = a['incid_hosp']/a['incid_hosp7']
a['location'] = 'France'
fig=coco.cocoa_basic_plot([a],['ratio'])
show(fig)